In [78]:
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from openai import OpenAI
import json
from healthcare_tools import tools_list, discharge_patient, get_patient_status
from langgraph.prebuilt import ToolNode


In [79]:
# Tools
def discharge_patient(state):
    """Discharge a patient from the hospital given their patient ID.
    
    Args:
        state: A dictionary containing patient_id and other state information
    
    Returns:
        The updated state dictionary
    """
    patient_id = state['patient_id']
    print("discharge_patient")
    print(state) 
    discharge_patient(patient_id)  
    return state


def get_patient_info(state):
    """Retrieve patient information given their patient ID.
    
    Args:
        state: A dictionary containing patient_id and other state information
    
    Returns:
        The updated state dictionary
    """
    patient_id = state['patient_id']
    print("get_patient_info")
    print(state)
    get_patient_status(patient_id)
    return state    

def tools_condition(state):
    print("are_tools_needed")
    
    # Get the tool name from the last assistant message
    messages = state.get('messages', [])
    if messages and hasattr(messages[-1], 'tool_calls'):
        tool_calls = messages[-1].tool_calls
        if tool_calls:
            tool_name = tool_calls[0].function.name
            if tool_name == 'discharge_patient':
                return 'discharge_patient'
            elif tool_name == 'get_patient_info':
                return 'get_patient_info'
    return END


In [80]:

client = OpenAI()

def call_open_ai (messages):
    response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools_list
    )
    return response



def admin_assistant(state):
    print ("admin_assistant")

    print (state)   
    messages = state['messages']
    response = call_open_ai(messages)
    messages.append(response.choices[0].message)
    state['messages'] = messages
    return state
   
    

In [81]:
from langchain.tools import Tool
# tools = [
#     Tool(
#         name="discharge_patient",
#         func=discharge_patient,
#         description="Discharge a patient from the hospital given their patient ID"
#     ),
#     Tool(
#         name="get_patient_info",
#         func=get_patient_info,
#         description="Retrieve patient information given their patient ID"
#     )
# ]
tools = [discharge_patient, get_patient_info]
# Set up the workflow
workflow = StateGraph(MessagesState)
workflow.add_node("admin_assistant", admin_assistant)
workflow.add_node("tools", ToolNode(tools))

# Set up the edges
workflow.add_edge(START, "admin_assistant")
workflow.add_conditional_edges(
    "admin_assistant",
    tools_condition,
    {
        "discharge_patient": "tools",
        "get_patient_info": "tools"
    }
)
workflow.add_edge("tools", "admin_assistant")

# Compile the graph
workflows_app = workflow.compile()

In [74]:
from langgraph.prebuilt import ToolNode
from langchain.tools import Tool

tools = [discharge_patient, get_patient_info]
# tools = [
#     Tool(
#         name="discharge_patient",
#         func=discharge_patient,
#         description="Discharge a patient from the hospital given their patient ID"
#     ),
#     Tool(
#         name="get_patient_info",
#         func=get_patient_info,
#         description="Retrieve patient information given their patient ID"
#     )
# ]

print ("tools_list", tools_list)

workflow = StateGraph(MessagesState)
# Add the initial system message node
workflow.add_node("admin_assistant", admin_assistant)
# workflow.add_node("tools", ToolNode(tools_list))
workflow.add_node("tools", ToolNode(tools))


# Set the entry point
# workflow.set_entry_point("admin_assistant")
workflow.add_edge(START, "admin_assistant")
workflow.add_conditional_edges("admin_assistant", tools_condition)
workflow.add_edge("discharge_patient", END)
workflow.add_edge(tools, "admin_assistant")


# Compile the graph
workflows_app = workflow.compile()

display(Image(workflows_app.get_graph().draw_mermaid_png()))

tools_list [{'type': 'function', 'function': {'name': 'discharge_patient', 'description': 'Discharge a Patient', 'parameters': {'type': 'object', 'properties': {'patient_id': {'type': 'string', 'description': 'Patient ID for the patient'}}, 'required': ['patient_id']}}}, {'type': 'function', 'function': {'name': 'get_patient_status', 'description': 'Provide status of the patient', 'parameters': {'type': 'object', 'properties': {'patient_id': {'type': 'string', 'description': 'Patient ID for the patient'}}, 'required': ['patient_id']}}}]


TypeError: unhashable type: 'Tool'

In [51]:
# user_input = "I need help discharging a patient. Can you assist me with that process?"
user_input = "What's the largest limb in the body?"
messages = [
    # System message defines the AI's role and available tools
    {"role": "system", "content": "You are a helpful assistant. Either answer a question or invoke the following tools:"\
          "discharge_patient(patient_id)"\
          "get_patient_status(patient_id)"
    },
    # Add the user's input to the conversation
    {"role": "user", "content": user_input}
] 
workflows_app.invoke(messages)  

InvalidUpdateError: Expected dict, got [{'role': 'system', 'content': 'You are a helpful assistant. Either answer a question or invoke the following tools:discharge_patient(patient_id)get_patient_status(patient_id)'}, {'role': 'user', 'content': "What's the largest limb in the body?"}]
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

In [52]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant. Either answer a question or invoke the following tools:discharge_patient(patient_id)get_patient_status(patient_id)'},
 {'role': 'user', 'content': "What's the largest limb in the body?"}]